In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from gensim import models
from tqdm.notebook import tqdm

In [2]:
def delete_punctuation_marks_tag(word):
    word = word.replace(",","")
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace("!","")
    word = word.replace(":","")
    word = word.replace("?","")
    word = word.replace("~","")
    word = word.replace("\\","")
    word = word.replace("\"","")
    word = word.replace(";","")
    return word.lower()

In [3]:
eat_info = pd.read_csv('../data/translated_eat_info.csv')
eat_review = pd.read_csv('../data/translated_eat_review.csv')

In [4]:
eat_review_by_user = pd.DataFrame(index = eat_review['u_id'].unique(), columns=['reviews'])

for id in tqdm(eat_review['u_id'].unique()):
    review_all = ''
    for review in eat_review.loc[eat_review['u_id']==id,'review_eng']:
        if type(review) != float:
            review_all += review+'. '
    eat_review_by_user.loc[id,'reviews'] = review_all

In [5]:
eat_info['tags']=''
eat_review_by_user['tags']=''

In [6]:
eat_info.head()

,name,address,category,main_mn,price,opng_tm,rating,rvw_cnt,tags,p_id,reviews
0,놀부만두,서울특별시 동대문구 휘경2동 276-33,NaN,냉모밀,6000.0,"{'월-금,일': '오전 11시 - 오후 8시 30분 '}",NaN,NaN,,721,"As a local restaurant, it was moderately shabb..."
1,민벅 건대점,서울특별시 광진구 화양동 10-1,NaN,알리오올리오,9900.0,{'매일': '오전 11시 30분 - 오후 10시 (구정 추석 당일 3시 오픈)'},NaN,NaN,,1832,"If you order 1 menu per person, give me Gorgon..."
2,뉴델리,서울시 동대문구 회기동 3-182,NaN,치킨 탄두리 (한마리),15000.0,{'매일': '오전 11시 - 오후 10시 '},NaN,NaN,,741,You can enjoy delicious Indian curry at an old...
3,강릉스낵,서울특별시 양천구 목동 406-65,NaN,직송 모듬 회,60000.0,"{'월-금': '오후 5시 - 오전 1시 ', '토/일': '오후 4시 - 오전 1...",NaN,NaN,,218,Tteokbokki is spicy and delicious. Fried chick...
4,여기가전집,서울특별시 구로구 구로3동 1124-34,NaN,모둠전,18000.0,{'월-금': '오후 2시 - 오전 2시 '},NaN,NaN,,3276,"The pancakes are delicious~ Potato pancake, pe..."


In [7]:
eat_review_by_user.head()

,reviews,tags
5351,"As a local restaurant, it was moderately shabb...",
3024,I went to Pyongyang naengmyeon in Jegi-dong an...,
5941,"Among the three, kimchi dumplings were the bes...",
13580,The combination of kimchi dumplings and cold b...,
6953,,


In [8]:
with open('../data/stop_words.txt','r',-1,'utf-8') as f:
    stop_words = set(f.read().split('/'))

In [ ]:
stop_words

{'',
 '*',
 '+',
 '+0',
 '+_+',
 '0th',
 '1',
 '1st',
 '2',
 '2nd',
 '3',
 '3rd',
 '4',
 '4b',
 '4th',
 '5th',
 '6',
 '6th',
 '7th',
 '8th',
 '9',
 '=',
 '>',
 '@',
 '__',
 '___________________________________',
 'a',
 'a4',
 'about',
 'above',
 'accumulated',
 'acrostic',
 'ade',
 'admirable',
 'adzuki',
 'after',
 'again',
 'against',
 'ago',
 'agre',
 'ah',
 'ahead',
 'ai',
 'ain',
 'ajumma',
 'aka',
 'al',
 'albert',
 'all',
 'alla',
 'alle',
 'alli',
 'allthe',
 'alma',
 'alongthe',
 'altro',
 'am',
 'amazing',
 'ame',
 'americano',
 'amo',
 'an',
 'anam',
 'ancestral',
 'ancient',
 'and',
 'anniversary',
 'annual',
 'ant',
 'anticipated',
 'any',
 'apo',
 'arabian',
 'archa',
 'are',
 'aren',
 "aren't",
 'argent',
 'arti',
 'as',
 'asu',
 'at',
 'attempted',
 'austrian',
 'avai',
 'average',
 'avo',
 'awesome',
 'b',
 'baby',
 'badam',
 'badi',
 'bahn',
 'bal',
 'baltic',
 'bam',
 'bb',
 'be',
 'bean',
 'beans',
 'because',
 'been',
 'beep',
 'bef',
 'before',
 'beforethe',
 'beg

In [ ]:
def get_tags_to_txt(text):
    if type(text) != str:
        return ''
    text = word_tokenize(text)
    tags_dump = nltk.pos_tag(text)
    tags = []
    
    for word in tags_dump:
        if (word[1] == 'JJ') and (word[0] not in stop_words):
            tags.append(delete_punctuation_marks_tag(word[0]))
            
    tags = ','.join(tags)
    
    return tags

In [ ]:
for i in tqdm(eat_info.index):
    eat_info.loc[i,'tags'] = get_tags_to_txt(eat_info.loc[i,'reviews'])
    
for i in tqdm(eat_review_by_user.index):
    eat_review_by_user.loc[i,'tags'] = get_tags_to_txt(eat_review_by_user.loc[i,'reviews'])

In [ ]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

In [ ]:
All_tags = set()
for tags in eat_info['tags']:
    All_tags.update(set(tags.split(',')))

In [ ]:
new_df = pd.DataFrame()
for word in All_tags:
    try:
        f_vec = w2v.get_vector(word)
        new_df[word] = f_vec
    except:
        pass

In [ ]:
from sklearn.cluster import KMeans
x = new_df.T
cluster_count=120
model = KMeans(n_clusters=cluster_count)
model.fit(x)
model.predict(x)
Y=x.copy()
Y['kmeans_id'] = model.predict(x)

In [ ]:
km = list(Y['kmeans_id'])
max_count = 0
max_count_id = ''
for i in range (0, cluster_count):
    if km.count(i) > max_count:
        max_count = km.count(i);
        max_count_id = i;
    print(i,': ', km.count(i))
max_count_id

In [ ]:
km_df = pd.DataFrame()
km_df['group_'+str(max_count_id)] = pd.Series(list(Y[Y['kmeans_id'] == max_count_id].index))
for i in range(0, cluster_count):
    col_name = 'group_'+str(i)
    words = list(Y[Y['kmeans_id'] == i].index)
    km_df[col_name] = pd.Series(words)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
km_df.fillna(0)

In [ ]:
# remove groups we don't need
# rm_groups = [17, 26, 50, 63]
# for i in rm_groups:
#     group_name = 'group_'+str(i);
#     stop_words.update(set(km_df[group_name].dropna()))
# with open('../data/stop_words.txt','w',-1,'utf-8') as f:
#     f.write('/'.join(stop_words))

In [ ]:
eat_info.set_index(eat_info['p_id'],inplace=True,drop=True)
del eat_info['p_id']
eat_info.head()

In [ ]:
res_matrix = pd.DataFrame(index=eat_info.index,columns=list(km_df.columns))
user_matrix = pd.DataFrame(index=eat_review_by_user.index,columns=list(km_df.columns))
res_matrix = res_matrix.fillna(0)
user_matrix = user_matrix.fillna(0)

In [ ]:
res_matrix.head()

In [ ]:
user_matrix.head()

In [ ]:
from collections import Counter

In [ ]:
for id in tqdm(res_matrix.index):
    tags = Counter(eat_info.loc[id,'tags'].split(','))
    for tag in tags.keys():
        for num in range(0,cluster_count):
            if tag in set(km_df['group_'+str(num)]):
                res_matrix.loc[id,'group_'+str(num)] += tags[tag]

In [ ]:
res_matrix.head()

In [ ]:
for id in tqdm(user_matrix.index):
    tags = Counter(eat_review_by_user.loc[id,'tags'].split(','))
    for tag in tags.keys():
        for num in range(0,cluster_count):
            if tag in set(km_df['group_'+str(num)]):
                user_matrix.loc[id,'group_'+str(num)] += tags[tag]

In [ ]:
user_matrix.head()

In [ ]:
user_matrix.head()

In [ ]:
km_df.count()

In [ ]:
eat_review_score = pd.DataFrame(eat_review, columns=['u_id','p_id','rating'])
eat_review_score.sort_values(by=['u_id'], axis=0,inplace=True)
eat_review_score.reset_index(drop = True, inplace=True)
eat_review_score.head()

In [ ]:
res_matrix.to_csv('../data/res_matrix.csv')
user_matrix.to_csv('../data/user_matrix.csv')
eat_review_score.to_csv('../data/score_board.csv',index=False)
km_df.count().to_csv('../data/group_count.csv')